In [642]:
from docscorer.configuration import ScorerConfiguration
from docscorer.docscorer import DocumentScorer
import json
import pandas as pd
config = ScorerConfiguration()
scorer = DocumentScorer(config)

path = "/mnt/d/Prompsit/WDS/aarepo_github/datos/pa_hplt.jsonl"
documents = []
i = 0
with open(path, "r", encoding="utf8") as a:
    for document in a:
        if i == 10000:
            break
        documents.append(json.loads(document))
        i+=1

In [643]:
mesured_docs = [ (scorer._extract_features(document_text=x["text"])["word_chars"], x["text"]) for x in documents]

In [73]:
def es_curva_gaussiana(lista, lim_inicial=14, tolerancia_ruido=0.1):
    import numpy as np

    # Debe tener al menos 5 valores
    if len(lista) < 5:
        return False
    
    arr = np.array(lista)

    # 1️⃣ Comienza con valores bajos (por debajo del límite)
    if arr[0] > lim_inicial:
        return False

    # 2️⃣ Encuentra el índice del máximo
    idx_max = np.argmax(arr)

    # 3️⃣ Verifica que los valores antes del máximo tienden a subir
    antes = arr[:idx_max]
    despues = arr[idx_max:]

    if len(antes) < 2 or len(despues) < 2:
        return False

    # Calcular correlación con una secuencia creciente/decreciente
    x_antes = np.arange(len(antes))
    x_despues = np.arange(len(despues))

    # Verificamos si "antes" es creciente y "después" decreciente
    correl_antes = np.corrcoef(x_antes, antes)[0, 1]
    correl_despues = np.corrcoef(x_despues, despues)[0, 1]

    # 4️⃣ Debe subir antes del máximo y bajar después
    if correl_antes < 0.5 or correl_despues > -0.5:
        return False

    # 5️⃣ Opcional: que la curva tenga una forma más o menos simétrica
    if abs(len(antes) - len(despues)) / len(lista) > 0.5:
        return False

    return True

In [71]:
gaussianas = [x for x in mesured_docs if es_curva_gaussiana(x[0])]

In [74]:
gaussianas2 = [x for x in mesured_docs if es_curva_gaussiana(x[0])]

In [75]:
len(gaussianas)

14

In [76]:
len(gaussianas2)

67

In [77]:
dif = [x for x in gaussianas2 if x not in gaussianas]

In [87]:
dif[9][1].split("\n")

['法律文书重要案件信息申诉审查',
 '您现在的位置:首页 - 检察动态 - 队伍建设',
 '荥经县检察院定期召开党课教育',
 '发布时间:2018/4/17 15:03:35  来源:本站  点击量:279',
 '近日,荥经县检察院召开党员大会,由支部副书记戴爱萍组织为全体党员上党课。',
 '首先,全体党员干部观看了《V观党课》,《V观党课》是由央视策划的一堂迷你党课,从12个标志性地点、12个不同身份党员,参加12场不同形式的党课,为你讲述“我的”一堂特别党课,观看后,结合学习习近平总书记来川视察重要讲话精神,组织了全体党员一起展开了讨论。',
 '会上周富林检察长要求:一是要抓好贯通结合,将学习贯彻习近平总书记来川视察重要讲话精神与“三会一课”有机结合起来;二是要按照全院检察工作绩效考核安排部署,以“清单制+责任制”工作法,逐条逐项梳理研究,落实工作责任,确保今年绩效考核工作圆满完成;三是院党组要做带头学习的表率楷模,院领导班子成员和各科室负责同志要先学一步、深学一层,原原本本、原汁原味学习习近平总书记重要讲话精神,不断推动工作上新台阶,力争进入全市考核第一方阵。',
 '上一条:荥经县检察院定期召开党课教育',
 '下一条:荥经县检察院 专题学习习近平总书记来川视察 重要讲话精神',
 '打印本页      关闭窗口',
 'Copyright (c) 2018 http://www.yayjjcy.gov.cn/    All Right Reserved.   蜀ICP备14008228号-1',
 '版权所有: 雅安市荥经县人民检察院       您是第55916位访客    今日访问251次    技术支持: 程友科技']

In [8]:
n=0
x = gaussianas[n][0]
print(gaussianas[n][1])

编辑: admin 来源: 狮威88 时间: 2018-8-22 6:15:43
酒店加盟,bet体育投注澳门体育投注足球外围网站哪个靠谱、伟德手机投注外围滚球手机客户端,澳门赌球网,北京早点小吃培训加盟 ,hg0088皇冠足球开户澳门永利网站澳门葡京线上赌球。
bet36体育在线投注365足球投注狮威88 赌球网论坛澳门网络现金赌球网广式早点加盟365体育投注官网在线新2网址,澳门赌球平台博狗体育手机投注 ,hg0088正网hg0088手机版网址最新赌球网,外围真钱赌球188体育投注澳门永利网站,狮威88 365体育投注备用网站新2网址备用。
365体育投注备用网站皇冠在线足球投注2018世界杯外围投注"早餐加盟品牌"bet36体育投注足球开户平台足球投注开户,最新赌球网站吉祥体育投注app 2018世界杯在哪投注澳门皇冠赌球网站澳门葡京线上赌球,伟德体育投注日博投注网外围足球开户金沙手机投注平台,bt365体育在线投注365外围赌球网站伟德体育手机投注。
体育投注网赌球网网址网上外围赌球网站新2网址备用 ,皇冠足球投注网址188亚洲体育足球在线开户,金沙足球开户网皇冠体育官网早餐店 加盟 手机外围滚球软件,明升体育投注葡京线上赌球网明升体育投注?hg0088官网现金赌球网。
网上赌球网站大全皇冠现金足球网站开户365娱乐场体育投注,足球投注开户网365体育投注平台网址bet36365体育在线投注365体育投注,365体育投注官方网足球外围网站哪个靠谱,皇冠开户投注网世界杯买球app足球外围投注app 足球网上投注皇冠现金手机投注网皇冠娱乐现金网,真钱赌球365娱乐场体育投注河南早点加盟 188体育投注澳门网上投注皇冠娱乐网址。
去澳门赌球网址真人365体育投注皇冠投注备用网址、足球现金投注导航网皇冠线上娱乐188投注,足球外围投注网皇冠新2手机版网址188体育平台澳门赌球玩法 金沙足彩投注金沙手机投注网,澳门金沙国际足球外围投注外围赌球网网址皇冠足球投注系统 皇冠足球外围投注网站网上赌球网站足球投注开户"河南早餐加盟"。
外围赌球平台皇冠娱乐场,葡京线上投注网站皇冠体育投注平台365体育投注网 澳门银河网上赌球365投注网在线赌球官网,博狗体育娱乐365体育投注澳门网络赌球网址,大发888体育投注大发888体育投注365投注网,澳门外围赌球澳门外围赌球浩博体育投

In [21]:
# n=0
n+=1
x = gaussianas[n][0]
average = (sum(x)/len(x) + 4 )/2
for i in range(len(gaussianas[n][0])):
    if gaussianas[n][0][i] < average:
        print(gaussianas[n][1].split("\n")[i])
    else:
        break
print("------------------")
for i in reversed(range(len(gaussianas[n][0]))):
    if gaussianas[n][0][i] < average:
        print(gaussianas[n][1].split("\n")[i])
    else:
        break

关于我们 加入我们 内容合作 版权声明 联系方式 免责条款 隐私政策 p2p资讯 p2p行业 p2p金融
“您好,我们是**银行下属贷款机构,请问您最近有贷款需求吗,月息五厘的信用贷款考不考虑?”
“您好,我们是**银行下属贷款机构,不上征信、随借随还、先息后本的信用贷款需要吗?利率北京地区最低。”
------------------


In [555]:
def short_segment_score(word_chars, menu_length, long_text_max):
        if max(word_chars) < long_text_max:
            return False
        import numpy as np
        num_segments = len(word_chars)
        # Almost 5 segments
        if num_segments < 5:
            return False
            
        arr = np.array(word_chars)
        idx_max = np.argmax(arr)
        before = arr[:idx_max]
        after = arr[idx_max:]
    
        if len(before) < 2 or len(after) < 2:
            return False
    
        # Correlation with a growing or decreasing sequence
        x_before = np.arange(len(before))
        x_after = np.arange(len(after))
    
        # before grows and after decrease
        if np.std(before) == 0 or np.std(after) == 0:
            return False
        correl_before = np.corrcoef(x_before, before)[0, 1]
        correl_after = np.corrcoef(x_after, after)[0, 1]
        if correl_before < 0.5 or correl_after > -0.5:
            return False
    
        # symmetry of the curve
        if abs(len(before) - len(after)) / len(word_chars) > 0.5:
            return False
        
        # At this point is a bell curve, now we determinate the number of possible boilerplate segments
        # before
        boilerplates = []
        for n in range(len(word_chars)):
            if word_chars[n] < menu_length * 2:
                boilerplates.append(n)
            else:
                break
        before_boilerplates = boilerplates if len(boilerplates) > 4 and sum(boilerplates)/len(boilerplates) <= menu_length else []
        
        # after
        boilerplates = []
        for n in reversed(range(len(word_chars))):
            if word_chars[n] < menu_length * 2:
                boilerplates.append(n)
            else:
                break
        after_boilerplates = boilerplates if len(boilerplates) > 4 and sum(boilerplates)/len(boilerplates) <= menu_length else []

        boilerplates = before_boilerplates+after_boilerplates
        if not boilerplates:
            return False
        
        num_boilerplates = len(boilerplates)
        
        score = (abs(num_boilerplates/num_segments - 1) + 3) / 4
        return (score, boilerplates)


    
    

In [644]:
with_menus = []
for n in range(len(mesured_docs)):
    x = short_segment_score(mesured_docs[n][0], 31, 260)
    if x:
        with_menus.append([mesured_docs[n], x])

In [645]:
len(with_menus)

326

In [665]:
# n=0
n+=1
text = with_menus[n][0][1].split("\n")
print(with_menus[n][1][0])
for x in with_menus[n][1][1]:
    print(text[x])

0.8043478260869565
ਜਦੋਂ ਕੁਲਵਿੰਦਰ ਬਿੱਲਾ ਦੇ ਐਕਸੀਡੈਂਟ ਦੀ ਖਬਰ ਨਾਲ ਦਰਸ਼ਕ ਹੋਏ ਪਰੇਸ਼ਾਨ – Desi News
Tuesday , September 27 2022
Terms and Conditions
Privacy Policy
About
Desi News Latest Information
Menu Item
Desi Treatment
Fun Time
News
Home/News/ਜਦੋਂ ਕੁਲਵਿੰਦਰ ਬਿੱਲਾ ਦੇ ਐਕਸੀਡੈਂਟ ਦੀ ਖਬਰ ਨਾਲ ਦਰਸ਼ਕ ਹੋਏ ਪਰੇਸ਼ਾਨ
ਜਦੋਂ ਕੁਲਵਿੰਦਰ ਬਿੱਲਾ ਦੇ ਐਕਸੀਡੈਂਟ ਦੀ ਖਬਰ ਨਾਲ ਦਰਸ਼ਕ ਹੋਏ ਪਰੇਸ਼ਾਨ
ਜਦੋਂ ਕੁਲਵਿੰਦਰ ਬਿੱਲਾ ਦੇ ਐਕਸੀਡੈਂਟ ਦੀ ਖਬਰ ਨਾਲ ਦਰਸ਼ਕ ਹੋਏ ਪਰੇਸ਼ਾਨ
© Copyright 2022, All Rights Reserved
Powered by WordPress | Designed by TieLabs
Twitter
Facebook
Share


In [263]:
text = with_menus[n][0][1].split("\n")
print(with_menus[n][1][0])
segments = ""
for x in with_menus[n][1][1]:
    segments = f"{segments}{text[x]}⏎"

0.9772727272727273


In [559]:
text

['Curry, John Steuart, 1897-1946Alternative names',
 'John Steuart Curry (1897-1946) was a painter, lithographer, and instructor.',
 'Curry worked as WPA muralist and is famous as one of The Regionalists, along with Thomas Hart Benton and Grant Wood. He taught at the University of Wisconsin, Madison, 1936-1946.',
 'From the description of John Steuart Curry and Curry family papers, 1848-1999. (Unknown). WorldCat record id: 233006998',
 'Painter, lithographer, instructor.',
 'Born in Kansas; worked as WPA muralist. Curry is famous as one of The Regionalists, along with Thomas Hart Benton and Grant Wood. Taught University of Wisconsin, Madison, 1936-',
 'From the description of John Steuart Curry and Curry family papers, 1897-1994. (Unknown). WorldCat record id: 220175371',
 "Painter, muralist, and illustrator John Steuart Curry is considered one of the three important painters of the American Regionalist movement, along with Thomas Hart Benton of Missouri and Grant Wood of Iowa. Curry w

In [667]:
with open("/mnt/d/Prompsit/WDS/aarepo_github/boilerplate_score/short_segments_pan.txt", "a+", encoding="utf8") as a:
    for n in range(len(with_menus)):
        text = with_menus[n][0][1].split("\n")
        segments = f"{round(with_menus[n][1][0],2)}\t"
        for x in with_menus[n][1][1]:
            segments = f"{segments}{text[x]}⏎"
        a.write(f"{segments}\n")

In [ ]:
# def short_segment_score(word_chars, menu_length):
#         import numpy as np
#         num_segments = len(word_chars)
#         # Almost 5 segments
#         if num_segments < 5:
#             return False
            
#         arr = np.array(word_chars)
#         idx_max = np.argmax(arr)
#         before = arr[:idx_max]
#         after = arr[idx_max:]
    
#         if len(before) < 2 or len(after) < 2:
#             return False
    
#         # Correlation with a growing or decreasing sequence
#         x_before = np.arange(len(before))
#         x_after = np.arange(len(after))
    
#         # before grows and after decrease
#         if np.std(before) == 0 or np.std(after) == 0:
#             return False
#         correl_before = np.corrcoef(x_before, before)[0, 1]
#         correl_after = np.corrcoef(x_after, after)[0, 1]
#         if correl_before < 0.5 or correl_after > -0.5:
#             return False
    
#         # symmetry of the curve
#         if abs(len(before) - len(after)) / len(word_chars) > 0.5:
#             return False
        
#         # At this point is a bell curve, now we determinate the number of possible boilerplate segments
        
#         average = (sum(word_chars)/len(word_chars) + menu_length)/2 #Average of all segments except for the biggest, and approached to the menu_length value using another average
#         boilerplates = []
        
#         # before
#         for n in range(len(word_chars)):
#             if (word_chars[n]+word_chars[n+1])/2 < average:
#                 boilerplates.append(n)
#             else:
#                 break
        
#         if len(boilerplates) < 5:
#             boilerplates = []
        
#         # after
#         for n in reversed(range(len(word_chars))):
#             if (word_chars[n]+word_chars[n-1])/2 < average:
#                 boilerplates.append(n)
#             else:
#                 break
#         num_boilerplates = len(boilerplates)
#         if num_boilerplates < 5:
#             return False
#         score = (abs(num_boilerplates/num_segments - 1) + 3) / 4
#         return (score, boilerplates)


    
    